#### Preprocessing Tox 21 Data

In [1]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Draw
import os
import glob
import numpy as np
import re
import xml.etree.ElementTree as ET
import json
from rdkit.Chem.Scaffolds import MurckoScaffold
from collections import defaultdict
import torch

##### Checking RDKit Applicability for Tox21 Molecules

In [2]:
# Load tox21 dataset
df = pd.read_csv("tox21.csv")

# check if RDKit can be applied to the molecules in the dataset
def apply_rdkit(smiles):
    return Chem.MolFromSmiles(smiles) is not None
    

total = len(df)
available = df["smiles"].apply(apply_rdkit).sum()

print(f"molecules with RDKit applicability: {available} / {total}")

[01:55:44] WARNING: not removing hydrogen atom without neighbors
[01:55:44] Explicit valence for atom # 8 Al, 6, is greater than permitted
[01:55:44] Explicit valence for atom # 3 Al, 6, is greater than permitted
[01:55:44] Explicit valence for atom # 4 Al, 6, is greater than permitted
[01:55:44] Explicit valence for atom # 4 Al, 6, is greater than permitted


molecules with RDKit applicability: 7823 / 7831


[01:55:44] Explicit valence for atom # 9 Al, 6, is greater than permitted
[01:55:44] Explicit valence for atom # 5 Al, 6, is greater than permitted
[01:55:44] Explicit valence for atom # 16 Al, 6, is greater than permitted
[01:55:45] Explicit valence for atom # 20 Al, 6, is greater than permitted


* Save the filtered dataset into refined_tox21.csv

In [3]:
valid_df = df[df["smiles"].apply(apply_rdkit)].copy()
valid_df.to_csv("refined_tox21.csv", index=False)

[01:55:50] WARNING: not removing hydrogen atom without neighbors
[01:55:50] Explicit valence for atom # 8 Al, 6, is greater than permitted
[01:55:50] Explicit valence for atom # 3 Al, 6, is greater than permitted
[01:55:50] Explicit valence for atom # 4 Al, 6, is greater than permitted
[01:55:50] Explicit valence for atom # 4 Al, 6, is greater than permitted
[01:55:50] Explicit valence for atom # 9 Al, 6, is greater than permitted
[01:55:50] Explicit valence for atom # 5 Al, 6, is greater than permitted
[01:55:50] Explicit valence for atom # 16 Al, 6, is greater than permitted
[01:55:50] Explicit valence for atom # 20 Al, 6, is greater than permitted


##### Creating a 13C-NMR Spectrum Dataset for Available Tox21 Molecules

* Converting SMILES Strings to InChIKeys for Spectral Data Matching

In [ ]:
mol_id_list = []
inchikey_list = []

for idx, row in valid_df.iterrows():
    mol_id = row['mol_id']
    smiles = row['smiles']
    
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        inchikey = Chem.MolToInchiKey(mol)
        
        # Only add to lists if InChIKey is valid
        if inchikey is not None:
            mol_id_list.append(mol_id)
            inchikey_list.append(inchikey)
        
    else:
        inchikey = None
        print(f"Invalid SMILES: {smiles}")

[23:40:20] WARNING: not removing hydrogen atom without neighbors


* Save the output as a CSV

In [5]:
# only leave mol id and inchikey
df_inchikey = pd.DataFrame({
    'mol_id': mol_id_list,
    'InChIKey': inchikey_list,
})

#remove duplicates
df_inchikey = df_inchikey.drop_duplicates(subset=['InChIKey'])

df_inchikey.to_csv('tox21_inchikey.csv', index=False)

In [6]:
len(df_inchikey)

7821

* Collecting 13C-NMR Spectral Data for Tox21 Molecules in NMRShiftDB2 database

In [7]:
db2_sdf_path = "NmrShift/nmrshiftdb2withsignals.sd" # path to NMRShiftdb2 SDF file
with open(db2_sdf_path, 'r', encoding='utf-8') as f:
    content = f.read()

# Split the content by "$$$$" to get individual molecules
molecules = content.split("$$$$")

In [8]:
# extract NMR spectra and InChI keys from NMRShift
spectra_dict = {}
for block in molecules:
    lines = block.splitlines()
    ik = None
    spec = None
    for i, line in enumerate(lines):
        line = line.strip()
        if line == "> <INChI key>":    
            if i + 1 < len(lines):
                ik = lines[i+1].strip()
        elif line == "> <Spectrum 13C 0>":
            if i + 1 < len(lines):
                spec = lines[i+1].strip()
        # end if both found
        if ik and spec:
            spectra_dict[ik] = spec
            break

print(f"total 13C NMR spectrum in NMRShift: {len(spectra_dict)}.")

total 13C NMR spectrum in NMRShift: 33005.


* Save each molecule’s binary spectrum vector, indicating presence (1) or absence (0) of 13C NMR peaks at 0.1 ppm intervals from –50 to 350 ppm, as a NumPy array in a npy file named by its molecule ID.

In [9]:
# parameters for binary vector
min_value, max_value, scale = -50.0, 350.0, 10  # 0.1 ppm intervals
units = int((max_value - min_value) * scale)   # 4000 dimension binary vector

spectra_dir = "spectra" # output directory for binary spectra
os.makedirs(spectra_dir, exist_ok=True)

# For each molecule, extract the InChIKey and corresponding spectrum
# convert the spectrum to a binary vector, and save it as a .npy file.
for _, row in df_inchikey.iterrows():
    ik     = row["InChIKey"]
    mol_id = row["mol_id"]
    spec_str = spectra_dict.get(ik)
    if not spec_str:
        continue

    # extract ppm values from the spectrum string
    indices = []
    for entry in spec_str.split("|"):
        if not entry: continue
        try:
            ppm = float(entry.split(";",1)[0]) # ppm value is before the first semicolon
        except ValueError:
            continue
        idx = int(round((ppm - min_value) * scale)) # convert ppm to index
        if 0 <= idx < units:
            indices.append(idx)

    if not indices:
        continue

    # create a binary vector of size 4000
    vec = np.zeros(units, dtype=np.uint8)
    vec[indices] = 1

    save_path = os.path.join(spectra_dir, f"{mol_id}.npy")
    np.save(save_path, vec)

print(f"Saved total {len(os.listdir(spectra_dir))} binary spectra to '{spectra_dir}'")

Saved total 2254 binary spectra to 'spectra'


* Collecting 13C-NMR Spectral Data for Tox21 Molecules in NP-MRD database

In [12]:
json_pattern = os.path.join("npmrd_natural_products_json", "*.json") # json file which contains InChIKey and NP-MRD ID(accession) of NP-MRD database
json_files = glob.glob(json_pattern)

# map inchikey to NP-MRD ID(accession)
inchikey2acc = {}
for js in json_files:
    with open(js, "r", encoding="utf-8") as f:
        data = json.load(f)
    for entry in data["np_mrd"]["natural_product"]:
        ik  = entry.get("inchikey")
        acc = entry.get("accession")
        if ik and acc:
            inchikey2acc[ik] = acc

df_inchikey["accession"] = df_inchikey["InChIKey"].map(inchikey2acc)

matched = df_inchikey["accession"].notna().sum()
print(f"{matched} NP-MRD compound data matched with Tox21 dataset.")

1551 NP-MRD compound data matched with Tox21 dataset.


In [13]:
existing = {
    os.path.splitext(fname)[0]
    for fname in os.listdir(spectra_dir)
    if fname.endswith(".npy")
}

df_acc = df_inchikey[df_inchikey["accession"].notna()]

# find which mol_id is already covered by existing spectra
covered = df_acc["mol_id"].astype(str).isin(existing)

# map mol_id to NP-MRD ID
acc2mol = dict(zip(df_inchikey["accession"], df_inchikey["mol_id"]))
to_add_acc = df_acc.loc[~covered, "accession"].unique() # list of NP-MRD IDs that need spectra

In [14]:
# list of directories which contain NP-MRD peak lists in txt format
configs = [
    ("NP-MRD_nmr_peak_lists/NP-MRD_nmr_peak_lists_*", "{acc}_*.txt"),
    ("NP-MRD_assignment_tables", "*_{acc}_*.txt"),
]

In [15]:
# iterate through the NP-MRD peak lists to find spectra
for acc in to_add_acc:
    for base_dir, file_pat in configs:
        pattern = os.path.join(base_dir, file_pat.format(acc=acc))
        files = glob.glob(pattern)
        if not files:
            continue

        indices = []
        with open(files[0], "r", encoding="utf-8") as f:
            for line in f:
                # conditions that contain 13C NMR spectra
                parts = re.split(r"[,\t]+", line.strip())
                if len(parts) < 3: # must have at least 3 columns
                    continue
                if parts[0] != "C" and parts[1] != "C": # not a carbon spectrum
                    continue
                ppm_str = parts[2] # ppm value is in the third column
                if ppm_str in ("", "NA"):
                    continue
                try:
                    ppm = float(ppm_str) # convert ppm string to float
                except ValueError:
                    continue

                idx = int(round((ppm - min_value) * scale)) # convert ppm index
                if 0 <= idx < units:
                    indices.append(idx)

        if not indices:
            continue

        # create a binary vector
        vec = np.zeros(units, dtype=np.uint8)
        vec[indices] = 1
        mol_id = acc2mol.get(acc, acc)
        save_path = os.path.join(spectra_dir, f"{mol_id}.npy") # save the binary vector as a .npy file
        np.save(save_path, vec)

In [16]:
for acc in to_add_acc:
    pattern = os.path.join("NP-MRD_peak_lists_unassigned", f"{acc}_*peak_list*.csv") # NP-MRD peak lists in CSV format
    files = glob.glob(pattern)
    if not files:
        continue

    txt_path = files[0]
    indices = []
    

    with open(txt_path, "r", encoding="utf-8") as f:
        for line in f:
            # conditions that contain 13C NMR spectra
            parts = re.split(r"[,\t ]+", line.strip())
            if len(parts) < 2:
                continue
            if parts[0] != "C": # first column must be 'C' for carbon spectrum
                continue
            ppm_str = parts[1] # ppm value is in the second column
            if ppm_str in ("", "NA"):
                continue
            try:
                ppm = float(ppm_str) # convert ppm string to float
            except ValueError:
                continue

            idx = int(round((ppm - min_value) * scale)) # convert ppm to index
            if 0 <= idx < units:
                indices.append(idx)

    if not indices:
        continue

    # create a binary vector
    vec = np.zeros(units, dtype=np.uint8)
    vec[indices] = 1

    mol_id = acc2mol.get(acc, acc)
    save_path = os.path.join(spectra_dir, f"{mol_id}.npy")
    np.save(save_path, vec)

print(f"Saved total {len(os.listdir(spectra_dir))} binary spectra to '{spectra_dir}'")

Saved total 2771 binary spectra to 'spectra'


* Collecting 13C-NMR Spectral Data for Tox21 Molecules in HMDB database

In [17]:
hmdb_sdf_path = "HMDB_structures/structures.sdf" # sdf file wich contains InChIKey and HMDB ID of HMDB database
with open(hmdb_sdf_path, "r", encoding="utf-8") as f:
    blocks = f.read().split("$$$$")

# map InChIKey to HMDB ID
inchikey2dbid = {}
for blk in blocks:
    lines = blk.splitlines()
    ik = dbid = None
    for i, L in enumerate(lines):
        L = L.strip()
        if L == "> <INCHI_KEY>" and i+1 < len(lines):
            ik = lines[i+1].strip()
        elif L == "> <DATABASE_ID>" and i+1 < len(lines):
            dbid = lines[i+1].strip()
        if ik and dbid:
            inchikey2dbid[ik] = dbid
            break

df_inchikey["HMDB_ID"] = df_inchikey["InChIKey"].map(inchikey2dbid)

In [18]:
# set of existing spectra after adding NP-MRD
existing = {
    os.path.splitext(f)[0]
    for f in os.listdir(spectra_dir)
    if f.endswith(".npy")
}

mask_hmdb   = df_inchikey["HMDB_ID"].notna()
mask_exists = df_inchikey["mol_id"].astype(str).isin(existing)
missing_spec = df_inchikey[mask_hmdb & ~mask_exists] # list of HDMB IDs that need spectra

In [19]:
# iterate through the missing HMDB spectra to find matching XML files
for _, row in missing_spec.iterrows():
    mol_id  = row["mol_id"]
    hmdb_id = row["HMDB_ID"]
    
    # search for XML files which match the HMDB IDs that need spectra
    pattern = os.path.join("hmdb_nmr_spectra", f"{hmdb_id}_nmr_one_d_spectrum_*.xml")
    files = glob.glob(pattern)
    if not files:
        continue

    shifts = []
    indices = []
    for xml_path in files:
        tree = ET.parse(xml_path)
        root = tree.getroot()
        notes = root.findtext("notes", default="").lower()
        if "13c spectrum" not in notes: # no 13C spectrum in this XML
            continue

        for peak in root.findall(".//nmr-one-d-peak"): # 13C spectrum peaks
            cs = peak.findtext("chemical-shift")
            if not cs:
                continue
            try:
                ppm = float(cs) # convert ppm to float
            except ValueError:
                continue
            shifts.append(ppm)
        break

    if not shifts:
        continue

    indices = [
        idx for ppm in shifts
        if 0 <= (idx := int(round((ppm - min_value) * scale))) < units # convert ppm to index
    ]

    if not indices:
        continue

    # create a binary vector
    vec = np.zeros(units, dtype=np.uint8)
    vec[indices] = 1
    save_path = os.path.join(spectra_dir, f"{mol_id}.npy")
    np.save(save_path, vec)

print(f"Saved total {len(os.listdir(spectra_dir))} binary spectra to '{spectra_dir}'")

Saved total 2851 binary spectra to 'spectra'


* These npy files represent the 1D NMR modality.

#### Create Molecular Image Dataset

* Generate 224×224 PNG images of molecules from SMILES strings in the refined Tox21 dataset using RDKit.

In [ ]:
img_dir = "images" # output directory for molecule images
os.makedirs(img_dir, exist_ok=True)
count = 0

# save images of molecules
for mol_id, smi in zip(valid_df["mol_id"], valid_df["smiles"]):
    mol = Chem.MolFromSmiles(smi)
    out_path = os.path.join(img_dir, f"{mol_id}.png")
    # skip if mol is None
    if mol is None:
        print(f"Invalid SMILES: {smi}")
        continue
    Draw.MolToFile(mol, out_path, size=(224, 224))
    count += 1
    
print(f"total {count} molecules saved to {img_dir}")

[19:39:51] WARNING: not removing hydrogen atom without neighbors


total 7823 molecules saved to images


* These png files represent the 2D CNN modality.

##### Generate train, validation, test splits using scaffold, and corresponding spectral datasets for each split.

* Replace missing labels with –1 to facilitate masking.

In [5]:
labels = [
    "NR-AR","NR-AR-LBD","NR-AhR","NR-Aromatase",
    "NR-ER","NR-ER-LBD","NR-PPAR-gamma",
    "SR-ARE","SR-ATAD5","SR-HSE","SR-MMP","SR-p53"
]
valid_df[labels] = valid_df[labels].fillna(-1)

In [9]:
# Function to generate scaffold from SMILES
def generate_scaffold(smiles, include_chirality=False):
    scaffold = MurckoScaffold.MurckoScaffoldSmiles(
        smiles=smiles, includeChirality=include_chirality)
    return scaffold

# Function to split dataset into train, validation, and test sets based on scaffold
def scaffold_split(index, smiles_list, frac_train=0.8, frac_valid=0.1, frac_test=0.1,
                                         sort=False, seed=None):
    """
    Split the dataset into train, validation, and test sets based on scaffold.
    Args:
        index: Sequence of indices for the dataset.
        smiles_list: List of SMILES strings corresponding to the dataset.
        frac_train/frac_valid/frac_test: Fractions that (approximately) sum to 1.0.
        sort: If True, sort split indices ascending before returning.
        seed: Random seed for reproducibility.
    """

    index = np.array(index)

    rng = np.random.default_rng(seed)

    # Build scaffold buckets
    scaffolds = defaultdict(list)
    for ind, smiles in enumerate(smiles_list):
        scaffold = generate_scaffold(smiles, include_chirality=False)
        scaffolds[scaffold].append(ind)

    # Shuffle the scaffold groups
    scaffold_sets = rng.permutation(np.array(list(scaffolds.values()), dtype=object))

    # target sizes
    n_total = len(index)
    n_total_train = int(np.floor(frac_train * n_total))
    n_total_valid = int(np.floor(frac_valid * n_total))
    n_total_test  = int(np.floor(frac_test  * n_total))

    train_idx, valid_idx, test_idx = [], [], []
    counts  = {"train": 0, "valid": 0, "test": 0}
    targets = {"train": n_total_train, "valid": n_total_valid, "test": n_total_test}

    # Assign each scaffold groups to the split with the largest deficit
    for g in scaffold_sets:
        deficits = {k: targets[k] - counts[k] for k in ("train", "valid", "test")}
        max_def = max(deficits.values())
        candidates = [k for k, v in deficits.items() if v == max_def]
        dest = rng.choice(candidates) # Randomly select a destination split if there is a tie
        if dest == "train":
            train_idx.extend(g); counts["train"] += len(g)
        elif dest == "valid":
            valid_idx.extend(g); counts["valid"] += len(g)
        else:
            test_idx.extend(g);  counts["test"]  += len(g)

    train_index, val_index, test_index = index[train_idx], index[valid_idx], index[test_idx]

    if sort:
        train_index = sorted(train_index)
        val_index = sorted(val_index)
        test_index = sorted(test_index)

    return train_index, val_index, test_index


In [27]:
# Generate scaffold-based splits
index = valid_df.index.values
smiles_list = valid_df["smiles"].tolist() 

train_idx, val_idx, test_idx = scaffold_split(
    index=index,
    smiles_list=smiles_list,
    frac_train=0.8,
    frac_valid=0.1,
    frac_test=0.1,
)

df_train = valid_df.loc[train_idx].reset_index(drop=True)
df_val   = valid_df.loc[val_idx].reset_index(drop=True)
df_test  = valid_df.loc[test_idx].reset_index(drop=True)

# Save the splits to CSV files
df_train.to_csv("train.csv", index=False)
df_val.to_csv("valid.csv", index=False)
df_test.to_csv("test.csv", index=False)

[13:34:56] WARNING: not removing hydrogen atom without neighbors


* Create train/valid/test data splits containing only molecules with available spectral data

In [28]:
# Create a set of mol_id values from the spectra directory
spectra_dir = "spectra"
all_specs = {
    os.path.splitext(fn)[0]
    for fn in os.listdir(spectra_dir)
    if fn.endswith(".npy")
}

In [29]:
# filter for mol_ids with available spectrum data
train_spectra_df = df_train[df_train["mol_id"].isin(all_specs)].copy()
train_spectra_df.to_csv("train_spectra.csv", index=False)
print(f"Saved train_spectra.csv: {len(train_spectra_df)}/{len(df_train)} samples")

valid_spectra_df = df_val[df_val["mol_id"].isin(all_specs)].copy()
valid_spectra_df.to_csv("valid_spectra.csv", index=False)
print(f"Saved valid_spectra.csv: {len(valid_spectra_df)}/{len(df_val)} samples")

test_spectra_df = df_test[df_test["mol_id"].isin(all_specs)].copy()
test_spectra_df.to_csv("test_spectra.csv", index=False)
print(f"Saved test_spectra.csv: {len(test_spectra_df)}/{len(df_test)} samples")

Saved train_spectra.csv: 2452/6258 samples
Saved valid_spectra.csv: 194/781 samples
Saved test_spectra.csv: 205/784 samples
